まずは真の報酬を与える

In [25]:

import numpy as np
from numpy import array
from scipy import io, sparse
import os
import sys # モジュール属性 argv を取得するため
import subprocess




waribiki=0.99

#行動1による報酬ベクトルr1を決定
r1=np.zeros(25)
for i in range(5):
    r1[i]=-1
r1[23]=10

#行動2による報酬ベクトルr2を決定
r2=np.zeros(25)
for i in range(5):
    r2[5*i]=-1
r2[19]=10

#行動3による報酬ベクトルr3を決定
r3=np.zeros(25)
for i in range(5):
    r3[i+20]=-1
r3[13]=10

#行動4による報酬ベクトルr4を決定
r4=np.zeros(25)
for i in range(5):
    r4[5*i+4]=-1
r4[17]=10

print(r1)
print(r2)
print(r3)
print(r4)

[-1. -1. -1. -1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0. 10.  0.]
[-1.  0.  0.  0.  0. -1.  0.  0.  0.  0. -1.  0.  0.  0.  0. -1.  0.  0.
  0. 10. -1.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 10.  0.  0.  0.  0.
  0.  0. -1. -1. -1. -1. -1.]
[ 0.  0.  0.  0. -1.  0.  0.  0.  0. -1.  0.  0.  0.  0. -1.  0.  0. 10.
  0. -1.  0.  0.  0.  0. -1.]


行動による状態遷移確率を書く

In [42]:
from numpy import *
from scipy import io, sparse

#行動１による状態遷移確率行列
A1=sparse.lil_matrix((25,25))
for i in range(5):
    A1[i,i]=1
for i in range(20):
    A1[i+5,i]=1
A1[23,18]=0
A1[23,1]=1


#行動２による状態遷移確率行列
A2=sparse.lil_matrix((25,25))
for i in range(25):
    if i%5==0:
        A2[i,i]=1
    if i%5>0:
        A2[i,i-1]=1
A2[19,18]=0
A2[19,1]=1

#行動3による状態遷移確率行列
A3=sparse.lil_matrix((25,25))
for i in range(5):
    A3[i+20,i+20]=1    
for i in range(20):
    A3[i,i+5]=1
A3[13,18]=0
A3[13,1]=1

#行動4による状態遷移確率行列
A4=sparse.lil_matrix((25,25))
for i in range(25):
    if i%5==4:
        A4[i,i]=1
    if i%5<4:
        A4[i,i+1]=1
A4[17,18]=0
A4[17,1]=1



In [27]:
print(A2)

  (0, 0)	1.0
  (1, 0)	1.0
  (2, 1)	1.0
  (3, 2)	1.0
  (4, 3)	1.0
  (5, 5)	1.0
  (6, 5)	1.0
  (7, 6)	1.0
  (8, 7)	1.0
  (9, 8)	1.0
  (10, 10)	1.0
  (11, 10)	1.0
  (12, 11)	1.0
  (13, 12)	1.0
  (14, 13)	1.0
  (15, 15)	1.0
  (16, 15)	1.0
  (17, 16)	1.0
  (18, 17)	1.0
  (19, 1)	1.0
  (20, 20)	1.0
  (21, 20)	1.0
  (22, 21)	1.0
  (23, 22)	1.0
  (24, 23)	1.0


In [28]:
#初期方策
from numpy.random import *
V=randn(25)
num=1

In [43]:
#iteration

num=num+1 #イテレーション回数

#行動価値関数つくる
Q1=r1+waribiki*A1*V
Q2=r2+waribiki*A2*V
Q3=r3+waribiki*A3*V
Q4=r4+waribiki*A4*V

#行動価値観数に基づいて方策改善
pi1=np.zeros(25)
pi2=np.zeros(25)
pi3=np.zeros(25)
pi4=np.zeros(25)
for i in range(25):
    pi1[i]=1*(Q1[i]==max(Q1[i],Q2[i],Q3[i],Q4[i]))
    pi2[i]=1*(Q2[i]==max(Q1[i],Q2[i],Q3[i],Q4[i]))*(1-pi1[i])
    pi3[i]=1*(Q3[i]==max(Q1[i],Q2[i],Q3[i],Q4[i]))*(1-pi1[i])*(1-pi2[i])
    pi4[i]=1-pi1[i]-pi2[i]-pi3[i]

 
P1 = sparse.lil_matrix((25,25))
P1.setdiag( pi1) 
P2 = sparse.lil_matrix((25,25))
P2.setdiag( pi2 ) 
P3 = sparse.lil_matrix((25,25))
P3.setdiag( pi3 ) 
P4 = sparse.lil_matrix((25,25))
P4.setdiag( pi4 ) 

print("pi created")

teisu=P1*r1+P4*r4+P3*r3+P2*r2
print("teisu-ko created")
E = sparse.lil_matrix((25,25))
x=np.ones(25)
E.setdiag(x)
keisu=E-waribiki*(P1*A1+P2*A2+P3*A3+P4*A4)
keisu=keisu.tocsr()
print("uhenkeisu created")


#keisu＊V＝teisuなので、両辺にkeisu-1を掛ければ良い。
keisu2=keisu.todense()
keisu_inv=np.linalg.inv(keisu2)

V=np.dot(keisu_inv,teisu)
V=np.squeeze(np.asarray(V))
pi=pi1+2*pi2+3*pi3+4*pi4
print(pi[0:5])
print(pi[5:10])
print(pi[10:15])
print(pi[15:20])
print(pi[20:25])
print(num)

pi created
teisu-ko created
uhenkeisu created
[3. 3. 3. 3. 3.]
[3. 3. 3. 3. 3.]
[3. 3. 3. 3. 2.]
[4. 4. 4. 1. 2.]
[1. 1. 1. 1. 1.]
9


In [44]:
#iteration
for q in range(100):
    num=num+1 #イテレーション回数

#行動価値関数つくる
    Q1=r1+waribiki*A1*V
    Q2=r2+waribiki*A2*V
    Q3=r3+waribiki*A3*V
    Q4=r4+waribiki*A4*V

#行動価値観数に基づいて方策改善
    pi1=np.zeros(25)
    pi2=np.zeros(25)
    pi3=np.zeros(25)
    pi4=np.zeros(25)
    for i in range(25):
        pi1[i]=1*(Q1[i]==max(Q1[i],Q2[i],Q3[i],Q4[i]))
        pi2[i]=1*(Q2[i]==max(Q1[i],Q2[i],Q3[i],Q4[i]))*(1-pi1[i])
        pi3[i]=1*(Q3[i]==max(Q1[i],Q2[i],Q3[i],Q4[i]))*(1-pi1[i])*(1-pi2[i])
        pi4[i]=1-pi1[i]-pi2[i]-pi3[i]

 
    P1 = sparse.lil_matrix((25,25))
    P1.setdiag( pi1) 
    P2 = sparse.lil_matrix((25,25))
    P2.setdiag( pi2 ) 
    P3 = sparse.lil_matrix((25,25))
    P3.setdiag( pi3 ) 
    P4 = sparse.lil_matrix((25,25))
    P4.setdiag( pi4 ) 

  
    teisu=P1*r1+P4*r4+P3*r3+P2*r2
    #print("teisu-ko created")
    E = sparse.lil_matrix((25,25))
    x=np.ones(25)
    E.setdiag(x)
    keisu=E-waribiki*(P1*A1+P2*A2+P3*A3+P4*A4)
    keisu=keisu.tocsr()
    #print("uhenkeisu created")


    #keisu＊V＝teisuなので、両辺にkeisu-1を掛ければ良い。
    keisu2=keisu.todense()
    keisu_inv=np.linalg.inv(keisu2)

    V=np.dot(keisu_inv,teisu)
    V=np.squeeze(np.asarray(V))
    pi=pi1+2*pi2+3*pi3+4*pi4
print(pi[0:5])
print(pi[5:10])
print(pi[10:15])
print(pi[15:20])
print(pi[20:25])
print(num)

[3. 3. 3. 3. 3.]
[3. 3. 3. 3. 3.]
[3. 3. 3. 3. 2.]
[4. 4. 4. 1. 2.]
[1. 1. 1. 1. 1.]
109


ここから逆強化学習

方策ベクトルを書く

In [45]:
Q2

array([191.09979759, 192.09979759, 194.04019958, 196.0002016 ,
       197.98000162, 193.04019958, 194.04019958, 196.0002016 ,
       197.98000162, 199.97979961, 195.0002016 , 196.0002016 ,
       197.98000162, 199.97979961, 201.99979759, 196.98000162,
       197.98000162, 199.97979961, 201.99979759, 204.04019958,
       195.0002016 , 196.0002016 , 197.98000162, 199.97979961,
       201.99979759])

In [46]:
V

array([194.04019958, 196.0002016 , 197.98000162, 199.97979961,
       197.98000162, 196.0002016 , 197.98000162, 199.97979961,
       201.99979759, 199.97979961, 197.98000162, 199.97979961,
       201.99979759, 204.04019958, 201.99979759, 199.97979961,
       201.99979759, 204.04019958, 201.99979759, 204.04019958,
       197.98000162, 199.97979961, 201.99979759, 204.04019958,
       201.99979759])